In [5]:
import numpy as np
import math
import residuals as r

In [59]:
# I am defining the weighted standard deviation according to this definition:
# https://en.wikipedia.org/wiki/Weighted_arithmetic_mean#Weighted_sample_variance
# I am using the inverse square of the error arrays to define the weights

def weighted_stddev(values, errors):
    '''
    Return the weighted average and standard deviation.

    '''
    weights = 1 / ((errors)**2)
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (math.sqrt(variance))

def jitter_ratio(allresids_file, dumpresids_file, condition_lower, condition_upper, titlearg):
    '''
    Return the ratio of sigma_dump over sigma_all.

    '''
    
    #Define sigma_all (or std_all)
    y=r.read_residuals(filename=allresids_file)
    FullResid = y.prefit_sec
    FullErrors = y.uncertainty
    std_all = weighted_stddev(FullResid, FullErrors)
    
    #=======
    
    #Define sigma_dump (or std_dump)
    x=r.read_residuals(filename=dumpresids_file)

    condition = (x.bary_TOA > condition_lower) & (x.bary_TOA < condition_upper)
    Resid = x.prefit_sec[condition]
    Errors = x.uncertainty[condition]
    std_dump = weighted_stddev(Resid, Errors)
    
    #Setup jitter ratio
    ratio = std_dump / std_all
    
    print titlearg
    print std_dump
    print "jitter ratio = " + str(ratio)

In [60]:
jitter_ratio("/nimrod1/eschwab/residuals/residJ1713_-56380.tmp",
             "/nimrod1/eschwab/residuals/resid_56380_80F8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 80 second subintervals')

J1713+0747, MJD 56380, 80 second subintervals
2.44415585816e-07
jitter ratio = 0.255675316386


In [61]:
jitter_ratio("/nimrod1/eschwab/residuals/residJ1713_-56380.tmp",
             "/nimrod1/eschwab/residuals/resid_56380_NTF8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 10 second subintervals')

J1713+0747, MJD 56380, 10 second subintervals
5.82671845721e-07
jitter ratio = 0.609514356488


In [62]:
jitter_ratio("/nimrod1/eschwab/residuals/residJ1713_-56201.tmp",
             "/nimrod1/eschwab/residuals/resid_56201_80F8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 80 second subintervals')

J1713+0747, MJD 56201, 80 second subintervals
1.73501235065e-07
jitter ratio = 0.336446382855


In [64]:
jitter_ratio("/nimrod1/eschwab/residuals/residJ1713_-56201.tmp",
             "/nimrod1/eschwab/residuals/resid_56201_NTF8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 10 second subintervals')

J1713+0747, MJD 56201, 10 second subintervals
5.43224438005e-07
jitter ratio = 1.05339824916


In [65]:
jitter_ratio("/nimrod1/eschwab/residuals/residB1937_-56465.tmp",
             "/nimrod1/eschwab/residuals/resid_56465_80F8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 80 second subintervals')

B1937+21, MJD 56465, 80 second subintervals
4.68059107539e-08
jitter ratio = 0.117597715425


In [66]:
jitter_ratio("/nimrod1/eschwab/residuals/residB1937_-56465.tmp",
             "/nimrod1/eschwab/residuals/resid_56465_NTF8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 10 second subintervals')

B1937+21, MJD 56465, 10 second subintervals
7.05138954322e-08
jitter ratio = 0.177162945341
